In [ ]:
# Importações para análise estatística
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# NLP Libraries
import nltk
from nltk import word_tokenize, sent_tokenize, ngrams
from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder
from nltk.corpus import stopwords
from textblob import TextBlob

# Visualizações
from wordcloud import WordCloud
import plotly.express as px
import plotly.graph_objects as go

# ML for similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Importar datasets
import sys
sys.path.append('..')
from datasets.textos_exemplo import *
from utils.nlp_utils import *

print("📊 Bibliotecas para análise estatística carregadas!")


In [ ]:
# Preparar corpus para análise
corpus = textos_noticias + textos_reviews + textos_academicos

# Tokenizar todo o corpus
all_tokens = []
for texto in corpus:
    tokens = word_tokenize(texto.lower())
    # Filtrar apenas palavras (remover pontuação)
    tokens = [token for token in tokens if token.isalpha()]
    all_tokens.extend(tokens)

print(f"📊 Corpus Statistics:")
print(f"Total de documentos: {len(corpus)}")
print(f"Total de tokens: {len(all_tokens):,}")
print(f"Vocabulário único: {len(set(all_tokens)):,}")
print(f"Type-Token Ratio: {len(set(all_tokens))/len(all_tokens):.4f}")

# Calcular frequências
freq_dist = Counter(all_tokens)

# Top 20 palavras mais frequentes
print("\n🔝 Top 20 palavras mais frequentes:")
for word, freq in freq_dist.most_common(20):
    print(f"{word:15}: {freq:6,} ({freq/len(all_tokens)*100:.2f}%)")

# Implementar análise da Lei de Zipf
def analyze_zipf_law(freq_dist, top_n=100):
    """Analisa se a distribuição segue a Lei de Zipf"""
    
    # Extrair frequências e rankings
    frequencies = [freq for word, freq in freq_dist.most_common(top_n)]
    ranks = list(range(1, len(frequencies) + 1))
    
    # Lei de Zipf: freq = C / rank
    # Em log: log(freq) = log(C) - log(rank)
    log_freq = np.log(frequencies)
    log_rank = np.log(ranks)
    
    # Regressão linear para estimar o expoente
    slope, intercept, r_value, p_value, std_err = stats.linregress(log_rank, log_freq)
    
    print(f"\n📈 Análise da Lei de Zipf (top {top_n} palavras):")
    print(f"Expoente (slope): {slope:.3f}")
    print(f"R²: {r_value**2:.3f}")
    print(f"Lei de Zipf ideal: slope = -1.0")
    print(f"Desvio da lei ideal: {abs(slope + 1):.3f}")
    
    return frequencies, ranks, slope, r_value**2

frequencies, ranks, slope, r2 = analyze_zipf_law(freq_dist)

# Visualizar Lei de Zipf
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Distribuição de frequências (escala normal)
ax1.plot(ranks[:50], frequencies[:50], 'bo-', alpha=0.7)
ax1.set_xlabel('Ranking da Palavra')
ax1.set_ylabel('Frequência')
ax1.set_title('Distribuição de Frequências (Top 50)')
ax1.grid(True, alpha=0.3)

# Plot 2: Lei de Zipf (escala log-log)
ax2.loglog(ranks, frequencies, 'ro-', alpha=0.7, label='Dados observados')

# Linha teórica da Lei de Zipf
zipf_theoretical = frequencies[0] / np.array(ranks)
ax2.loglog(ranks, zipf_theoretical, 'b--', alpha=0.7, label='Lei de Zipf ideal')

ax2.set_xlabel('Ranking (log)')
ax2.set_ylabel('Frequência (log)')
ax2.set_title(f'Lei de Zipf (R² = {r2:.3f})')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Análise de hapax legomena (palavras que aparecem apenas uma vez)
hapax_count = sum(1 for freq in freq_dist.values() if freq == 1)
hapax_ratio = hapax_count / len(freq_dist)

print(f"\n🔤 Análise de Hapax Legomena:")
print(f"Palavras com frequência 1: {hapax_count:,}")
print(f"Proporção de hapax: {hapax_ratio:.3f}")
print(f"Exemplos de hapax: {list(freq_dist.keys())[:10] if hapax_count > 0 else 'Nenhum'}")
